# **Welcome to the Python Priority Objectives Report Automation Tool!**
## Please enure you ***do not*** have a previous version of the PowerPoint open before running.
## Start by selecting 'Kernel' -> 'Restart Kernel and Run All Cells...'
## After running the kernel, in the prompt displayed below please type the quarter you wish to display in your PowerPoint:
#### Please type in q1, q2, q3, q4, or all

In [ ]:
try:
    import pandas as pd
    import re
    from IPython.display import FileLink, display
    from pptx import Presentation
    from pptx.util import Pt, Inches
    from pptx.dml.color import RGBColor
except ImportError:
    print("The 'pptx' library is required to run this code. Installing...")
    !pip install python-pptx
    # Now attempt to import again after installation
    try:
        import pandas as pd
        import re
        from IPython.display import FileLink, display
        from pptx import Presentation
        from pptx.util import Pt, Inches
        from pptx.dml.color import RGBColor
    except ImportError:
        raise ImportError("Failed to import 'pptx' library even after installation.")
    else:
        print("Successfully installed and imported 'pptx' library.")


def clean_data(data):
    # Grabs columns needed for powerpoint
    data = data[['Objective #', 'Section', 'Strategic Goal', 'Priority Objective', 'Quarterly Status']]
    df = data.astype(str)
    
    # Rename and format the columns to align with previous reports
    df['Objective #'] = df['Objective #'] + ' - ' + df['Section']
    df.rename({'Objective #': 'Objective # / Section', 'Priority Objective': 'FY2024 Priority Objective', 'Quarterly Status': 'FY2024 Metric'}, axis=1, inplace=True)
    df.drop(columns=['Section'], inplace=True)
    df = df.replace(r'\n', ' ', regex=True)
    return df

def clean_metric(df, quarter):
    q = ['q1', 'Q1', 'q2', 'Q2', 'q3', 'Q3', 'q4', 'Q4']
    # Extract only specified quarter Status from FY2024 Metric column using regular expressions
    if quarter in q:
        quarter_upper = quarter.upper()
        num = int(quarter_upper[1]) + 1
        df['FY2024 Metric'] = df['FY2024 Metric'].apply(lambda x: re.search(r'('+quarter_upper+' Status:\s*.*?)\s*(Q'+str(num)+' Status:|$)', x).group(1) if re.search(r'('+quarter_upper+' Status:\s*.*?)\s*(Q'+str(num)+' Status:|$)', x) else '')
    return df

def create_slide(slide, data, strategic_goal):
    # Creates title
    shapes = slide.shapes
    title_shape = shapes.title
    title_shape.text = strategic_goal
    p = title_shape.text_frame.paragraphs[0]
    p.font.size = Pt(20)
    p.font.bold = True
    p.font.name = 'Verdana'
    
    # Ensures maximum of 6 rows
    data = data.head(5)

    # Edits position of table
    rows, cols = data.shape[0] + 1, data.shape[1]
    left = Inches(0.25)
    top = Inches(1.2)
    width = Inches(9.5)
    height = Inches(2)
    table = slide.shapes.add_table(rows, cols, left, top, width, height).table
    font_size = 7.5
    white_fill = RGBColor(255, 255, 255)
    ice_blue_fill = RGBColor(218, 222, 229)
    fill_color = ice_blue_fill

    # Creates table
    for i, column in enumerate(data.columns):
        cell = table.cell(0, i)
        cell.text = column
        para = cell.text_frame.paragraphs[0]
        para.font.size = Pt(font_size)
        para.font.name = 'Verdana'
        if len(data) == 1:
            fill_color = white_fill

        for j, row in enumerate(data.values):
            if len(data) > 1:
                fill_color = white_fill if fill_color == ice_blue_fill else ice_blue_fill
            for i, value in enumerate(row):
                cell = table.cell(j + 1, i)
                cell.text = str(value)
                para = cell.text_frame.paragraphs[0]
                para.font.size = Pt(font_size)
                para.font.name = 'Verdana'
                cell.fill.solid()
                cell.fill.fore_color.rgb = fill_color

def main():
    # Reads in excel spreadsheet
    data = pd.read_excel('priorities.xlsx')
    
    # Clean data
    df = clean_data(data)
    
    # Prompt user for quarter selection
    quarters = ['q1', 'Q1', 'q2', 'Q2', 'q3', 'Q3', 'q4', 'Q4', 'all', 'All', 'ALL']
    while True:
        quarter = str(input('Please enter what quarter you would like to be displayed in your powerpoint: '))
        if quarter in quarters:
            break
        else:
            print("Invalid Input, please try again")
    
    # Clean metric based on selected quarter
    df = clean_metric(df, quarter)
    
    # Group by Strategic Goal
    grouped = df.groupby('Strategic Goal')
    
    # Initialize PowerPoint presentation
    prs = Presentation('template.pptx')
    slide_layout = prs.slide_layouts[3] 
    
    # Deletes empty content textbox from slide layout
    for shape in slide_layout.placeholders:
        if shape.placeholder_format.idx == 1:  
            sp = shape
            slide_layout.placeholders._spTree.remove(sp._element)
    
    # Iterates through the strategic goals and creates slides
    title_top = Inches(1.1)
    for strategic_goal, data in grouped:
        while len(data) > 0:
            slide = prs.slides.add_slide(slide_layout)
            create_slide(slide, data, strategic_goal)
            slide.shapes.title.top = title_top
            slide.shapes.title.left = Inches(.25)
            data = data.iloc[5:]
    
    # Save PowerPoint presentation
    prs.save('priorityObjectivesReport.pptx')
    
    # Display download link for the presentation
    display(FileLink('priorityObjectivesReport.pptx', result_html_prefix="Click here to download: "))

if __name__ == "__main__":
    main()
